In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import matplotlib.pyplot as plt
from PIL import Image
import imageio
import io
%matplotlib inline

data = pd.read_csv('podatki-smrti-statRegije.csv',delimiter=';', skiprows=[0,1], encoding='windows-1252')
data = data.head(13)
data = data.drop(data.columns[0], axis=1)
data.replace('...', 0, inplace=True)
data = data.rename(columns=lambda x: x.split()[0])

Razlika (v procentih) smrti dveh let po statističnih regijah.

In [2]:
slovenia = gpd.read_file('stanford-kt691jt0328-shapefile/kt691jt0328.shp')
slovenia['name_1'] = slovenia['name_1'].str.replace('?', 'š')
slovenia['name_1'] = slovenia['name_1'].str.replace('Spodnjeposavska', 'Posavska')
slovenia['name_1'] = slovenia['name_1'].str.replace('Notranjsko-kraška', 'Primorsko-notranjska')

merged = slovenia.merge(data, left_on="name_1", right_on="STATISTIÈNA")
diff = merged.copy()


def make_image(year):
    val = []
    if year != '2002':
        prev_year = str(int(year) - 1)
        for i, row in merged.iterrows():
            val.append((row[year] - row[prev_year]) / row[prev_year] * 100)
        else:
            diff[year] = val

    ax = diff.plot(column=year, cmap='coolwarm', legend=True)
    ax.set_title(f"Razlike[%] smrti po regijah med {prev_year} - {year}")
    ax.set_axis_off()
    fig = ax.get_figure()

    buffer = io.BytesIO()
    fig.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image = Image.open(buffer)

    plt.close()

    return image

images = []
for i in range(2003, 2022):
    images.append(make_image(str(i)))

imageio.mimsave("animated_map.gif", images, fps=1)

from IPython.display import Image

with open('animated_map.gif','rb') as f:
    display(Image(data=f.read(), format='gif'))


In [3]:
data = pd.read_excel('Eu annual crude death rate.xlsx')

eu_drzave = data.iloc[:, 0].tolist()[1:]
years = data.columns[1:].tolist()
death_rates = data.iloc[1:, 1:].apply(pd.to_numeric, errors='coerce')

death_rates.index = eu_drzave
death_rates = death_rates.dropna()
average_death_rates = death_rates.mean(axis=1)

izbrane_drzave = ['Slovenia']
lowest_2_countries = average_death_rates.nsmallest(2).index.tolist()
highest_2_countries = average_death_rates.nlargest(2).index.tolist()
izbrane_drzave += lowest_2_countries + highest_2_countries


slovenia_avg = average_death_rates.loc['Slovenia']
similar_countries = average_death_rates[abs(average_death_rates - slovenia_avg) < 0.5].index.tolist()
izbrane_drzave += similar_countries[:2]

death_rates_filter = death_rates.loc[izbrane_drzave]


plt.figure(figsize=(12, 6))
for country in izbrane_drzave:
    plt.plot(years, death_rates_filter.loc[country, :], label=country, linewidth=2)

plt.xticks(rotation=90
plt.xlabel('Year')
plt.ylabel('Število smrt na 1000 prebivalcev')
plt.title('Število smrt na 1000 prebivalcev v izbranih EU državah')

plt.legend(loc='upper left')
plt.tight_layout()
plt.show()



In [ ]:
from PIL import Image

europe = gpd.read_file("europeShp/Europe.shp")

crude_death_rates = pd.read_excel("Eu annual crude death rate.xlsx", index_col=0)

merged_data = europe.merge(crude_death_rates, left_on='NAME', right_index=True)

images = []
for year in range(2000, 2022):

    merged_data[str(year)].fillna(np.nan, inplace=True)
    ax = merged_data.plot(column=str(year), cmap='coolwarm', legend=False, missing_kwds={'color': 'gray'})

    ax.set_title(f'Crude Death Rate in Europe - {year}')
    ax.set_axis_off()
    fig = ax.get_figure()
    
    buffer = io.BytesIO()
    fig.savefig(buffer, format='png', bbox_inches='tight')
    buffer.seek(0)
    image = Image.open(buffer)
    plt.close()
    
    images.append(np.array(image))

imageio.mimsave('crude_death_rates.gif', images, fps=2)

from IPython.display import Image

with open('crude_death_rates.gif','rb') as f:
    display(Image(data=f.read(), format='gif'))